In [1]:
import numpy as np
import pandas as pd

In [2]:
ufo_data = pd.read_csv('complete.csv')
elect_pres_data = pd.read_csv('president-1976-2016.csv')

In [3]:
ufo_data.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [4]:
elect_pres_data.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party,writein,candidatevotes,totalvotes,version,notes
0,1976,Alabama,AL,1,63,41,US President,"Carter, Jimmy",democrat,False,659170,1182850,20171015,NaN
1,1976,Alabama,AL,1,63,41,US President,"Ford, Gerald",republican,False,504070,1182850,20171015,NaN
2,1976,Alabama,AL,1,63,41,US President,"Maddox, Lester",american independent party,False,9198,1182850,20171015,NaN
3,1976,Alabama,AL,1,63,41,US President,"Bubar, Benjamin """"Ben""""",prohibition,False,6669,1182850,20171015,NaN
4,1976,Alabama,AL,1,63,41,US President,"Hall, Gus",communist party use,False,1954,1182850,20171015,NaN


In [5]:
#clean election data to pull only the winning candidate for each state/year combination.
elect_pres_data['year_state_max'] = elect_pres_data.groupby(['year','state_po'])['candidatevotes'].transform('max')
elect_pres_data.drop(elect_pres_data[elect_pres_data['candidatevotes'] != elect_pres_data['year_state_max']].index, inplace = True)





In [6]:
#only rows showing the winning candidates remain
elect_pres_data.head()
elect_pres_data.dtypes

year                int64
state              object
state_po           object
state_fips          int64
state_cen           int64
state_ic            int64
office             object
candidate          object
party              object
writein              bool
candidatevotes      int64
totalvotes          int64
version             int64
notes             float64
year_state_max      int64
dtype: object

In [45]:
#cleaning ufo data
ufo_data.drop(ufo_data[ufo_data['country'] != 'us'].index, inplace = True)
ufo_data['state'] = ufo_data['state'].str.upper()

#split datetime column into two columns, date and time
ufo_data['date'], ufo_data['time'] = ufo_data['datetime'].str.split(' ',1).str
ufo_data['date']= pd.to_datetime(ufo_data['date'])  
ufo_data['year'] = pd.DatetimeIndex(ufo_data['date']).year
ufo_data['last_election_year'] = ufo_data['year'] - (ufo_data['year'] % 4) 

ufo_data.head()

<ipython-input-45-2cafb8ce51bc>:6: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  ufo_data['date'], ufo_data['time'] = ufo_data['datetime'].str.split(' ',1).str


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,date,time,year,last_election_year
0,10/10/1949 20:30,san marcos,TX,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,1949-10-10,20:30,1949,1948
3,10/10/1956 21:00,edna,TX,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,1956-10-10,21:00,1956,1956
4,10/10/1960 20:00,kaneohe,HI,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,1960-10-10,20:00,1960,1960
5,10/10/1961 19:00,bristol,TN,us,sphere,300,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595,-82.188889,1961-10-10,19:00,1961,1960
7,10/10/1965 23:45,norwalk,CT,us,disk,1200,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.1175,-73.408333,1965-10-10,23:45,1965,1964


In [46]:
#now add the relevant info from the us elect data 
ufo_data_merged = ufo_data.merge(elect_pres_data, how='outer', left_on=['last_election_year','state'], right_on=['year','state_po'])
ufo_data_merged.head()

,datetime,city,state_x,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,...,state_ic,office,candidate,party,writein,candidatevotes,totalvotes,version,notes,year_state_max
0,10/10/1949 20:30,san marcos,TX,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10/10/1956 21:00,edna,TX,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/15/1957 20:30,houston (ellington afb),TX,us,light,3,3 seconds,5 to 6&quot; diameter light&#44 extremely fast...,9/17/2000,29.7630556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11/10/1957 19:00,corpus christi,TX,us,light,600,10 minutes,UFO over Corpus Christi near naval yard on Nov...,10/31/2003,27.8002778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/1/1957 21:00,dallas,TX,us,unknown,20,20 seconds,I was flying the first of 4 USAF aircraft on a...,8/24/2004,32.7833333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
#create cleaned table for export

ufo_elect_cleaned = ufo_data_merged[['date','time','last_election_year','city','state_x','shape','duration (seconds)','duration (hours/min)','comments','date posted','latitude','longitude','candidate','party','candidatevotes','totalvotes']]

ufo_elect_cleaned.head()

,date,time,last_election_year,city,state_x,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,candidate,party,candidatevotes,totalvotes
0,1949-10-10,20:30,1948.0,san marcos,TX,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,NaN,NaN,NaN,NaN
1,1956-10-10,21:00,1956.0,edna,TX,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,NaN,NaN,NaN,NaN
2,1957-10-15,20:30,1956.0,houston (ellington afb),TX,light,3,3 seconds,5 to 6&quot; diameter light&#44 extremely fast...,9/17/2000,29.7630556,-95.363056,NaN,NaN,NaN,NaN
3,1957-11-10,19:00,1956.0,corpus christi,TX,light,600,10 minutes,UFO over Corpus Christi near naval yard on Nov...,10/31/2003,27.8002778,-97.396111,NaN,NaN,NaN,NaN
4,1957-01-01,21:00,1956.0,dallas,TX,unknown,20,20 seconds,I was flying the first of 4 USAF aircraft on a...,8/24/2004,32.7833333,-96.800000,NaN,NaN,NaN,NaN


In [10]:
#ufo_elect_cleaned.to_csv('merged_ufo_elect_data.csv')

In [40]:
#clean special characters from comments and create a column including comments in an array. 

def remove_s_characters(string_test):
    string_test_2 = ''
    for e in string_test:
        if e.isalnum():
            string_test_2 = string_test_2 + e
            
        elif e == ' ':
            string_test_2 = string_test_2 + e
    
    return string_test_2

def split_string(string):
    string_list = string.split()
    return string_list

ufo_elect_cleaned['comments'] = ufo_elect_cleaned['comments'].astype('str')

ufo_elect_cleaned['comments'] = ufo_elect_cleaned['comments'].apply(remove_s_characters)
ufo_elect_cleaned['comments_array'] = ufo_elect_cleaned['comments'].apply(split_string)


<ipython-input-40-c4d30427f63b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ufo_elect_cleaned['comments'] = ufo_elect_cleaned['comments'].astype('str')
<ipython-input-40-c4d30427f63b>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ufo_elect_cleaned['comments'] = ufo_elect_cleaned['comments'].apply(remove_s_characters)
<ipython-input-40-c4d30427f63b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [41]:
ufo_elect_cleaned.to_csv('ufo_elect_data_v2.csv')